# 01_Escolha_features_e_pipeline

## 01_import_bibliotecas

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import numpy as np

## 02_Carga_de_dados

In [ ]:
vagas = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/vagas.json"

In [ ]:
# vagas
response_vagas = requests.get(vagas)
data_vagas = response_vagas.json()

## 03_seleção_e_tratamento_de_features

In [ ]:

# === Carregar dados (ajuste o caminho conforme seu projeto local)
dados = pd.read_excel("dados.xlsx", sheet_name="dados", usecols=[
    "area_atuacao", "certificacoes", "data_atualizacao", "data_criacao",
    "nivel_academico", "nivel_espanhol", "nivel_ingles",
    "outras_certificacoes", "outro_idioma"
])

In [ ]:

# === Definir data de referência (hoje)
hoje = pd.to_datetime("today")


In [ ]:

# === Converter colunas de data e calcular deltas
dados["data_atualizacao"] = pd.to_datetime(dados["data_atualizacao"], errors='coerce')
dados["data_criacao"] = pd.to_datetime(dados["data_criacao"], errors='coerce')


In [ ]:

dados["dias_desde_atualizacao"] = (hoje - dados["data_atualizacao"]).dt.days
dados["dias_ativo"] = (hoje - dados["data_criacao"]).dt.days


In [ ]:

# === Criar colunas binárias
dados["tem_outras_certificacoes"] = dados["outras_certificacoes"].notnull().astype(int)
dados["tem_outro_idioma"] = dados["outro_idioma"].notnull().astype(int)


In [ ]:

# === Preencher vazios com string vazia para TF-IDF
dados["area_atuacao"] = dados["area_atuacao"].fillna("")
dados["certificacoes"] = dados["certificacoes"].fillna("")


In [ ]:

# === TF-IDF para campos multivalorados separados por vírgula
tfidf_area = TfidfVectorizer(tokenizer=lambda x: x.split(","), lowercase=True)
tfidf_cert = TfidfVectorizer(tokenizer=lambda x: x.split(","), lowercase=True)


In [ ]:

tfidf_area_matrix = tfidf_area.fit_transform(dados["area_atuacao"])
tfidf_cert_matrix = tfidf_cert.fit_transform(dados["certificacoes"])


In [ ]:

# === OneHot Encoding para níveis de idioma e formação
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
onehot_matrix = encoder.fit_transform(dados[[
    "nivel_academico", "nivel_espanhol", "nivel_ingles"
]].fillna(""))


In [ ]:

# === Combinar todas as variáveis em uma matriz final
final_matrix = hstack([
    tfidf_area_matrix,
    tfidf_cert_matrix,
    onehot_matrix,
    np.array(dados[[
        "dias_desde_atualizacao", "dias_ativo",
        "tem_outras_certificacoes", "tem_outro_idioma"
    ]])
])


In [ ]:

# === Criar DataFrame final com nomes das colunas
col_names = (
    [f"area_{feat}" for feat in tfidf_area.get_feature_names_out()] +
    [f"cert_{feat}" for feat in tfidf_cert.get_feature_names_out()] +
    list(encoder.get_feature_names_out([
        "nivel_academico", "nivel_espanhol", "nivel_ingles"
    ])) +
    ["dias_desde_atualizacao", "dias_ativo",
     "tem_outras_certificacoes", "tem_outro_idioma"]
)



In [ ]:
df_cluster_ready = pd.DataFrame(final_matrix.toarray(), columns=col_names)

# === (Opcional) salvar a base tratada
df_cluster_ready.to_csv("base_tratada_para_cluster.csv", index=False)